# Models

In [1]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
class BaselineModel(nn.Module):
    def __init__(self):
        super().__init__()
        # For 32x32 colour images
        self.conv_layer = nn.Conv2d(3, 32, (3, 3))
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        # ((32-2) * (32-2) * 32) / 4 = 7200
        self.dense = nn.Linear(7200, 128)
        # Relu after dense
        self.output = nn.Linear(128, 10) # 10 possible categories
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.to(device)
        x = self.conv_layer(x) # 32x32x3 -> 30x30x32
        x = self.relu(x) # 30x30x32 -> 30x30x32
        x = self.max_pool(x) # 30x30x32 -> 15x15x32
        x = self.flatten(x) # 15x15x32 -> 7200
        x = self.dense(x) # 7200 -> 128
        x = self.relu(x) # 128 -> 128
        x = self.output(x) # 128 -> 10
        x = self.softmax(x) # 10 -> 10
        return x


In [3]:
# It has one more convolutional layer than the original model
class ExtraConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        # For 32x32 colour images
        self.conv_layer = nn.Conv2d(3, 32, (3, 3))
        self.conv_layer_2 = nn.Conv2d(32, 32, (2, 2))
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(6272, 128)
        # Relu after dense
        self.output = nn.Linear(128, 10) # 10 possible categories
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.to(device)
        x = self.conv_layer(x) # 32x32x3 -> 30x30x32
        x = self.conv_layer_2(x) # 30x30x32 -> 29x29x32
        x = self.relu(x)
        x = self.max_pool(x) # 14x14x32
        x = self.flatten(x)
        x = self.dense(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

In [4]:
# It has two more convolutional layers than the original model
class ExtraDoubleConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        # For 32x32 colour images
        self.conv_layer = nn.Conv2d(3, 32, (3, 3))
        self.conv_layer_2 = nn.Conv2d(32, 64, (4, 4))
        self.conv_layer_3 = nn.Conv2d(64, 32, (2, 2))
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(5408, 128)
        # Relu after dense
        self.output = nn.Linear(128, 10) # 10 possible categories
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.to(device)
        x = self.conv_layer(x) # 32x32x3 -> 30x30x32
        x = self.conv_layer_2(x) # 30x30x32 -> 29x29x32
        x = self.conv_layer_3(x)
        x = self.relu(x)
        x = self.max_pool(x) # 14x14x32
        x = self.flatten(x)
        x = self.dense(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

In [5]:
# It's like the ExtraConv model, but it has a dropout layer between the two convolutions
class DropoutModel(nn.Module):
    def __init__(self):
        super().__init__()
        # For 32x32 colour images
        self.conv_layer = nn.Conv2d(3, 32, (3, 3))
        self.dropout = nn.Dropout(p=0.5)
        self.conv_layer_2 = nn.Conv2d(32, 32, (2, 2))
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(6272, 128)
        # Relu after dense
        self.output = nn.Linear(128, 10) # 10 possible categories
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.to(device)
        x = self.conv_layer(x) # 32x32x3 -> 30x30x32
        x = self.dropout(x)
        x = self.conv_layer_2(x) # 30x30x32 -> 29x29x32
        x = self.relu(x)
        x = self.max_pool(x) # 14x14x32
        x = self.flatten(x)
        x = self.dense(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

# Helper Functions

In [6]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
from tqdm import tqdm

In [7]:
# To have the data in a more PyTorch-friendly format
class CustomCIFAR10Dataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [8]:
# train_batches -> [batch_num][b'data'][image_num][x][y][rgb]
# test_batches -> [image_num][b'data'][x][y][rgb]
def load_cifar10_data(data_dir):
    train_batches = [unpickle(data_dir + '/data_batch_' + str(i)) for i in range(1, 6)]
    for batch in train_batches:
        batch[b'data'] = batch[b'data'].reshape((10000, 3, 32, 32))  # Reshape to (num_samples, channels, height, width)
        batch[b'data'] = batch[b'data'].astype('float32') / 255.0  # Convert to float and normalize

    test_batch = unpickle(data_dir + '/test_batch')
    test_batch[b'data'] = test_batch[b'data'].reshape((10000, 3, 32, 32))  # Reshape to (num_samples, channels, height, width)
    test_batch[b'data'] = test_batch[b'data'].astype('float32') / 255.0  # Convert to float and normalize

    return train_batches, test_batch

In [9]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [10]:
def see_sample(train_dataset, index):
    image, label = train_dataset[index]

    print(f"Image shape: {image.shape}")
    print(f"Label: {label}")

    # Visualize the image
    plt.imshow(image)  # Permute the dimensions for visualization
    plt.axis('off')
    plt.title(f"Label: {label}")
    plt.show()

# Train and test

In [11]:
def train(model, train_dataloader, epochs=100, lr=0.001):
    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # For viewing the training process in console
    progress_bar = tqdm(range(epochs), desc="Training", unit="epoch")

    # Train the model
    for epoch in progress_bar:
        for i, (images, labels) in enumerate(train_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # Forward pass
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize

In [12]:
def train_with_val(model, train_dataloader, val_dataloader, epochs=100, lr=0.001):
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    progress_bar = tqdm(range(epochs), desc="Training", unit="epoch")
    accuracy = None

    for epoch in progress_bar:
        model.train()
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            accuracy = correct / total

    print()
    print(f"Validation Accuracy: {accuracy:.4f}")

    # Print the validation accuracy

In [13]:
def test(model, test_dataloader):
    # Define the loss function
    criterion = nn.CrossEntropyLoss()

    # Set the model to evaluation mode
    model.eval()

    # Initialize variables to track test accuracy and loss
    correct = 0
    total = 0
    test_loss = 0.0

    # For viewing the testing process in console
    progress_bar = tqdm(test_dataloader, desc="Testing", unit="batch")

    # Disable gradient calculation for testing
    with torch.no_grad():
        for images, labels in progress_bar:
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Calculate loss
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Update accuracy statistics
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average test loss and accuracy
    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct / total

    # Print the test results
    print()
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# **MAIN FUNCTION**

In [14]:
# Get the default training and testing batches given by the dataset itself
train_batches, test_batch = load_cifar10_data("cifar-10-batches-py")

# Pass the data to a more PyTorch-friendly format
train_dataset = CustomCIFAR10Dataset(
    torch.cat([torch.tensor(batch[b'data']) for batch in train_batches]),
    torch.cat([torch.tensor(batch[b'labels']) for batch in train_batches]))
test_dataset = CustomCIFAR10Dataset(test_batch[b'data'], test_batch[b'labels'])

import random
import numpy as np
import torch

# Set the random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# Create custom-sized batches
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)


In [29]:
# Train and test the base model
print("BASE MODEL RESULTS:")
model = BaselineModel().to(device)
train(model, train_dataloader)
test(model, test_dataloader)

BASE MODEL RESULTS:


Testing: 100%|██████████| 79/79 [00:00<00:00, 446.75batch/s]


Test Loss: 1.8228, Test Accuracy: 0.6342


In [30]:
# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model2 = ExtraConvModel().to(device)
train(model2, train_dataloader)
test(model2, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Testing: 100%|██████████| 79/79 [00:00<00:00, 390.61batch/s]


Test Loss: 1.8147, Test Accuracy: 0.6476


In [31]:
# Train and test the double extra convolution model
print()
print("DOUBLE EXTRA CONVOLUTION MODEL RESULTS:")
model3 = ExtraDoubleConvModel().to(device)
train(model3, train_dataloader)
test(model3, test_dataloader)


DOUBLE EXTRA CONVOLUTION MODEL RESULTS:


Testing: 100%|██████████| 79/79 [00:00<00:00, 213.48batch/s]


Test Loss: 1.8627, Test Accuracy: 0.5982


In [15]:
# Train and test the dropout model
print()
print("DROPOUT EXTRA CONVOLUTION MODEL RESULTS:")
model4 = DropoutModel().to(device)
train(model4, train_dataloader)
test(model4, test_dataloader)


DROPOUT EXTRA CONVOLUTION MODEL RESULTS:


Testing: 100%|██████████| 79/79 [00:00<00:00, 322.33batch/s]


Test Loss: 1.8369, Test Accuracy: 0.6217


# HYPERPARAMETER TUNING

## Batch size

In [16]:
from torch.utils.data import random_split

In [17]:
# Bigger batch size
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_a = ExtraConvModel().to(device)
train_with_val(model_a, train_dataloader, val_dataloader)
# test(model_a, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [02:18<00:00,  1.39s/epoch]


Validation Accuracy: 0.6344


In [18]:
# Smaller batch size
batch_size = 64
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_b = ExtraConvModel().to(device)
train_with_val(model_b, train_dataloader, val_dataloader)
# test(model_b, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [03:19<00:00,  2.00s/epoch]


Validation Accuracy: 0.6196


## Learning rate

In [19]:
# Bigger learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_c = ExtraConvModel().to(device)
train_with_val(model_c, train_dataloader, val_dataloader, lr=0.005)
# test(model_c, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [02:15<00:00,  1.36s/epoch]


Validation Accuracy: 0.0971


In [20]:
# (A little) bigger learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_d = ExtraConvModel().to(device)
train_with_val(model_d, train_dataloader, val_dataloader, lr=0.002)
# test(model_d, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [02:18<00:00,  1.39s/epoch]


Validation Accuracy: 0.6396


## Epochs

In [21]:
# Smaller learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_e = ExtraConvModel().to(device)
train_with_val(model_e, train_dataloader, val_dataloader, lr=0.0005, epochs=200)
# test(model_e, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 200/200 [04:38<00:00,  1.39s/epoch]


Validation Accuracy: 0.6219


In [22]:
# Smaller learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_f = ExtraConvModel().to(device)
train_with_val(model_f, train_dataloader, val_dataloader, lr=0.0008, epochs=200)
# test(model_f, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 200/200 [04:39<00:00,  1.40s/epoch]


Validation Accuracy: 0.6290


In [23]:
# Smaller learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_g = ExtraConvModel().to(device)
train_with_val(model_g, train_dataloader, val_dataloader, lr=0.0015, epochs=200)
# test(model_g, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 200/200 [04:36<00:00,  1.38s/epoch]


Validation Accuracy: 0.6327


# Batch normalisation and data augmentation

In [24]:
class ExtraConvModelWithBN(nn.Module):
    def __init__(self):
        super().__init__()
        # For 32x32 colour images
        self.conv_layer = nn.Conv2d(3, 32, (3, 3))
        self.bn1 = nn.BatchNorm2d(32)
        self.conv_layer_2 = nn.Conv2d(32, 32, (2, 2))
        self.bn2 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.dense = nn.Linear(6272, 128)
        self.bn3 = nn.BatchNorm1d(128)
        # Relu after dense
        self.output = nn.Linear(128, 10)  # 10 possible categories
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.to(device)
        x = self.conv_layer(x)  # 32x32x3 -> 30x30x32
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv_layer_2(x)  # 30x30x32 -> 29x29x32
        x = self.bn2(x)
        x = self.relu(x)
        x = self.max_pool(x)  # 14x14x32
        x = self.flatten(x)
        x = self.dense(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.softmax(x)
        return x

In [25]:
# (A little) bigger learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_bn = ExtraConvModelWithBN().to(device)
train_with_val(model_bn, train_dataloader, val_dataloader, lr=0.002)
# test(model_bn, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [03:06<00:00,  1.87s/epoch]


Validation Accuracy: 0.6839


With both data augmentation and batch normalization: h

In [26]:
import torchvision.transforms as T
from torch.utils.data import ConcatDataset

# (A little) bigger learning rate
batch_size = 256
train_split, val_split = random_split(train_dataset, [0.8, 0.2])

train_split = ConcatDataset([
    [(T.Compose([
        T.RandomHorizontalFlip(),
        T.RandomCrop(32, padding=4),
    ])(x), y) for x, y in train_split] for _ in range(5)
])

# train_split = [(T.Compose([
#     T.ToTensor(),
#     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])(x), y) for x, y in train_split]

train_dataloader = DataLoader(train_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
val_dataloader = DataLoader(val_split, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, worker_init_fn=seed_worker)

# Train and test the extra convolution model
print()
print("EXTRA CONVOLUTION MODEL RESULTS:")
model_aug = ExtraConvModelWithBN().to(device)
train_with_val(model_aug, train_dataloader, val_dataloader, lr=0.002)
# test(model_aug, test_dataloader)


EXTRA CONVOLUTION MODEL RESULTS:


Training: 100%|██████████| 100/100 [13:52<00:00,  8.33s/epoch]


Validation Accuracy: 0.7256


In [27]:
test(model_a, test_dataloader)
test(model_b, test_dataloader)
test(model_c, test_dataloader)
test(model_d, test_dataloader)
test(model_e, test_dataloader)
test(model_f, test_dataloader)
test(model_g, test_dataloader)
test(model_bn, test_dataloader)
test(model_aug, test_dataloader)

Testing: 100%|██████████| 40/40 [00:00<00:00, 174.53batch/s]



Test Loss: 1.8276, Test Accuracy: 0.6312


Testing: 100%|██████████| 40/40 [00:00<00:00, 186.32batch/s]



Test Loss: 1.8464, Test Accuracy: 0.6134


Testing: 100%|██████████| 40/40 [00:00<00:00, 185.38batch/s]


...


Test Loss: 1.8289, Test Accuracy: 0.6292


Testing: 100%|██████████| 40/40 [00:00<00:00, 183.01batch/s]



Test Loss: 1.7829, Test Accuracy: 0.6791


Testing: 100%|██████████| 40/40 [00:00<00:00, 180.28batch/s]


Test Loss: 1.7335, Test Accuracy: 0.7244
